# Start

In [1]:
import pandas as pd
xls = pd.ExcelFile('Judgment4.xlsx')
df_screen = pd.read_excel(xls, 'Screen')
df_a = pd.read_excel(xls, 'a')
df_b = pd.read_excel(xls, 'b')
df_c = pd.read_excel(xls, 'c')

In [2]:
df_a = df_a.dropna()
df_b = df_b.dropna()
df_c = df_c.dropna()

In [3]:
df_a['fldMaR'] = df_a['fldMaR'].apply(lambda s: s.lower())

In [4]:
df_a['fldMaR'] = df_a['fldMaR'].str.replace("\(p\)", ",")
df_a['fldMaR'] = df_a['fldMaR'].str.replace("\(q\)", ",")
df_a['fldMaR'] = df_a['fldMaR'].str.replace(" q ", ",")
df_a['fldMaR'] = df_a['fldMaR'].str.replace(" p ", ",")
df_a['fldMaR'] = df_a['fldMaR'].str.replace("\(\)", ",")

/var/folders/nh/bqvjwbm11q39ckgjc3_j5_h80000gn/T/ipykernel_45836/855549382.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_a['fldMaR'] = df_a['fldMaR'].str.replace("\(p\)", ",")
/var/folders/nh/bqvjwbm11q39ckgjc3_j5_h80000gn/T/ipykernel_45836/855549382.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_a['fldMaR'] = df_a['fldMaR'].str.replace("\(q\)", ",")
/var/folders/nh/bqvjwbm11q39ckgjc3_j5_h80000gn/T/ipykernel_45836/855549382.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_a['fldMaR'] = df_a['fldMaR'].str.replace("\(\)", ",")


In [86]:
df_a

,fldMaR,fldMaS
0,call and apologize,2.0
1,make a phone call,2.0
2,atempt to call and say you would be late.,2.0
3,try to call.,2.0
4,call someone and tell them i would be late.,2.0
...,...,...
952,"i would be frantic,i'd get up and get my cloth...",1.0
953,call and tell them i'll be late.,2.0
954,i would go into a panic. get up and either tel...,2.0
955,call the place and see if i can reschedule or ...,2.0


In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_a['fldMaR'], df_a['fldMaS'], test_size = 0.05)

In [88]:
X_train

70                         hurry around and get ready , 
550    call them up and tell them i forgot and will b...
175                                     call,say be late
542    make a phone call and beg for forgiveness that...
305                                   call and tell them
                             ...                        
851    i would immediately call who i had the appt wi...
315                        call and say going to be late
955    call the place and see if i can reschedule or ...
113    i would call them and cancel or tell them that...
341                   call and tell that i had a problem
Name: fldMaR, Length: 909, dtype: object

In [89]:
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

In [90]:
X_train

,fldMaR
70,"hurry around and get ready ,"
550,call them up and tell them i forgot and will b...
175,"call,say be late"
542,make a phone call and beg for forgiveness that...
305,call and tell them
...,...
851,i would immediately call who i had the appt wi...
315,call and say going to be late
955,call the place and see if i can reschedule or ...
113,i would call them and cancel or tell them that...


In [91]:
y_train

,fldMaS
70,1.0
550,2.0
175,2.0
542,2.0
305,2.0
...,...
851,2.0
315,2.0
955,2.0
113,2.0


In [92]:
#X_train_backup, X_test_backup, y_train_back_up, y_test_backup = X_train, X_test, y_train, y_test

In [93]:
#X_train, X_test, y_train, y_test = X_train_backup, X_test_backup, y_train_back_up, y_test_backup

In [94]:
#from sentence_transformers import SentenceTransformer
#bert_model = SentenceTransformer('saved_models/bert.pb')

In [8]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer


def embed(texts):
    # Import our models. The package will take care of downloading the models automatically
    tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
    model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

    # Get the embeddings
    with torch.no_grad():
        embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

    return embeddings

In [9]:
embed("hi").shape

torch.Size([1, 1024])

In [96]:
#from sentence_transformers import SentenceTransformer
#bert_model2 = SentenceTransformer('distilbert-base-uncased')

In [97]:
#import tensorflow_hub as hub

#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [98]:
lst_X_train = X_train['fldMaR'].tolist()
#X_train_embedding_tensor = bert_model.encode(lst_X_train, convert_to_tensor=True)
X_train_embedding_tensor = embed(lst_X_train)

In [99]:
X_train_embeddings = []

for e in X_train_embedding_tensor:
    X_train_embeddings.append(e.tolist())

In [100]:
df_X_train_embeddings = pd.DataFrame(columns=['fldMaR'])
df_X_train_embeddings['fldMaR'] = X_train_embedding_tensor.squeeze().tolist()

In [101]:
df_X_train_embeddings

,fldMaR
0,"[-0.3672241270542145, -0.896980881690979, 0.04..."
1,"[-0.40602999925613403, -0.8170621395111084, -0..."
2,"[-0.478663831949234, -0.28665512800216675, -0...."
3,"[-0.4598325788974762, -0.4606322944164276, -0...."
4,"[-0.381252259016037, -0.5550910830497742, 0.55..."
...,...
904,"[-0.14179180562496185, 0.29456406831741333, 0...."
905,"[-0.2911086082458496, -0.1730719804763794, -0...."
906,"[0.14192607998847961, -0.23637710511684418, 0...."
907,"[-0.11157785356044769, 0.11203710734844208, -0..."


In [102]:
#X_train_embeddings = []
#for e in X_train_embedding_tensor:
#    X_train_embeddings.append(list(e))

In [103]:
#df_X_train_embeddings = pd.DataFrame(out, columns=["fldSResponse"])

In [104]:
lst_X_test = X_test['fldMaR'].tolist()
#X_test_embedding_tensor = bert_model.encode(lst_X_test, convert_to_tensor=True)
X_test_embedding_tensor = embed(lst_X_test)

In [105]:
X_test_embeddings = []

for e in X_test_embedding_tensor:
    X_test_embeddings.append(e.tolist())

In [106]:
df_X_test_embeddings = pd.DataFrame(columns=['fldMaR'])
df_X_test_embeddings['fldMaR'] = X_test_embedding_tensor.squeeze().tolist()

In [107]:
df_X_test_embeddings

,fldMaR
0,"[-0.2893258333206177, -0.013050932437181473, 0..."
1,"[-0.5309541821479797, -0.6432598233222961, 0.1..."
2,"[0.7075684070587158, -0.5955490469932556, 0.14..."
3,"[-0.6947242617607117, -0.781536340713501, 0.54..."
4,"[-0.06241545081138611, -0.3602924048900604, -0..."
5,"[-0.077024444937706, -0.5150719285011292, -0.5..."
6,"[-0.5280616879463196, -0.6082912683486938, -0...."
7,"[-0.5890246033668518, -0.6096842885017395, 0.1..."
8,"[0.014528116211295128, -0.14018568396568298, -..."
9,"[-0.650503396987915, 0.12910747528076172, -0.1..."


In [108]:
#X_test_embeddings = []
#for e in X_test_embedding_tensor:
#    X_test_embeddings.append(list(e))

In [109]:
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [110]:
X_train_embeddings_backup, X_test_embeddings_backup = X_train_embeddings, X_test_embeddings

In [111]:
type(X_train_embeddings_backup)

list

In [112]:
X_train_embeddings_backup = pd.DataFrame(X_train_embeddings_backup)
X_test_embeddings_backup = pd.DataFrame(X_test_embeddings_backup)

In [113]:
import numpy as np

In [114]:
X_train_embeddings = np.array(X_train_embeddings)
X_test_embeddings = np.array(X_test_embeddings)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [115]:
y_test

array([[0.],
       [2.],
       [0.],
       [1.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],
       [2.],
       [1.],
       [2.],
       [1.],
       [2.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [2.],
       [0.],
       [2.],
       [2.],
       [0.],
       [1.],
       [2.],
       [1.],
       [1.],
       [1.],
       [2.],
       [1.]])

In [116]:
import ast 
import boto3
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from datetime import date, timedelta
import datetime
import numpy
import os
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
import tensorflow as tf
import tensorflow.keras
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical


histories = []
preds = []


checkpoint_filepath = './tmp/checkpoint_bahar_a5'
model_checkpoint_callback = ModelCheckpoint(
      filepath=checkpoint_filepath,
      monitor='val_loss',
      verbose=1,
      save_best_only=True)

for batch in [16, 32, 64, 128]:
    inputs = Input(shape=(len(X_train_embeddings[0]),))
    model = Dense(64, activation='relu')(inputs)
    model = Dropout(0.2)(model)
    model = Dense(32, activation='tanh')(model)
    model = Dropout(0.2)(model)
    output = Dense(3, activation='softmax')(model)
    model = Model(inputs, output)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=5e-4)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    y_train_onehot = to_categorical(y_train, num_classes=3)
    y_test_onehot = to_categorical(y_test, num_classes=3)
    history = model.fit(
        X_train_embeddings, y_train_onehot, 
        epochs=100, batch_size=batch,
        validation_data=(X_test_embeddings, y_test_onehot),
        callbacks=[model_checkpoint_callback], shuffle=True)
    histories.append(history)

# to get predictions in the format [[2.], [0.], [2.], ...]
predictions = np.argmax(model.predict(X_test_embeddings), axis=-1).reshape(-1, 1)


Epoch 1/100
57/57 [==============================] - 0s 4ms/step - loss: 0.6612 - accuracy: 0.7305 - val_loss: 0.5158 - val_accuracy: 0.7917

Epoch 00001: val_loss improved from inf to 0.51576, saving model to ./tmp/checkpoint_bahar_a5
INFO:tensorflow:Assets written to: ./tmp/checkpoint_bahar_a5/assets
Epoch 2/100
57/57 [==============================] - 0s 3ms/step - loss: 0.5407 - accuracy: 0.7932 - val_loss: 0.5332 - val_accuracy: 0.7917

Epoch 00002: val_loss did not improve from 0.51576
Epoch 3/100
57/57 [==============================] - 0s 3ms/step - loss: 0.4956 - accuracy: 0.8020 - val_loss: 0.5051 - val_accuracy: 0.7708

Epoch 00003: val_loss improved from 0.51576 to 0.50508, saving model to ./tmp/checkpoint_bahar_a5
INFO:tensorflow:Assets written to: ./tmp/checkpoint_bahar_a5/assets
Epoch 4/100
57/57 [==============================] - 0s 3ms/step - loss: 0.4445 - accuracy: 0.8361 - val_loss: 0.5189 - val_accuracy: 0.7500

Epoch 00004: val_loss did not improve from 0.50508
Ep


Epoch 00041: val_loss did not improve from 0.45701
Epoch 42/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0741 - accuracy: 0.9758 - val_loss: 0.8352 - val_accuracy: 0.7917

Epoch 00042: val_loss did not improve from 0.45701
Epoch 43/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0919 - accuracy: 0.9681 - val_loss: 0.7835 - val_accuracy: 0.7708

Epoch 00043: val_loss did not improve from 0.45701
Epoch 44/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0694 - accuracy: 0.9769 - val_loss: 0.8424 - val_accuracy: 0.7917

Epoch 00044: val_loss did not improve from 0.45701
Epoch 45/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0720 - accuracy: 0.9813 - val_loss: 0.7200 - val_accuracy: 0.8125

Epoch 00045: val_loss did not improve from 0.45701
Epoch 46/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0796 - accuracy: 0.9692 - val_loss: 0.8330 - val_accuracy: 0.8333

Epoch 00046: val_loss did not

57/57 [==============================] - 0s 3ms/step - loss: 0.0456 - accuracy: 0.9824 - val_loss: 1.1197 - val_accuracy: 0.8542

Epoch 00084: val_loss did not improve from 0.45701
Epoch 85/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0460 - accuracy: 0.9824 - val_loss: 1.0476 - val_accuracy: 0.8333

Epoch 00085: val_loss did not improve from 0.45701
Epoch 86/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0450 - accuracy: 0.9835 - val_loss: 1.0819 - val_accuracy: 0.8333

Epoch 00086: val_loss did not improve from 0.45701
Epoch 87/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0651 - accuracy: 0.9780 - val_loss: 1.1160 - val_accuracy: 0.8333

Epoch 00087: val_loss did not improve from 0.45701
Epoch 88/100
57/57 [==============================] - 0s 3ms/step - loss: 0.0412 - accuracy: 0.9835 - val_loss: 1.1046 - val_accuracy: 0.8333

Epoch 00088: val_loss did not improve from 0.45701
Epoch 89/100
57/57 [=======================

29/29 [==============================] - 0s 4ms/step - loss: 0.1357 - accuracy: 0.9582 - val_loss: 0.5904 - val_accuracy: 0.7917

Epoch 00026: val_loss did not improve from 0.45283
Epoch 27/100
29/29 [==============================] - 0s 4ms/step - loss: 0.1322 - accuracy: 0.9604 - val_loss: 0.6214 - val_accuracy: 0.7708

Epoch 00027: val_loss did not improve from 0.45283
Epoch 28/100
29/29 [==============================] - 0s 4ms/step - loss: 0.1193 - accuracy: 0.9747 - val_loss: 0.6075 - val_accuracy: 0.7708

Epoch 00028: val_loss did not improve from 0.45283
Epoch 29/100
29/29 [==============================] - 0s 4ms/step - loss: 0.1288 - accuracy: 0.9582 - val_loss: 0.7273 - val_accuracy: 0.7500

Epoch 00029: val_loss did not improve from 0.45283
Epoch 30/100
29/29 [==============================] - 0s 4ms/step - loss: 0.1066 - accuracy: 0.9714 - val_loss: 0.6483 - val_accuracy: 0.7917

Epoch 00030: val_loss did not improve from 0.45283
Epoch 31/100
29/29 [=======================

15/15 [==============================] - 0s 5ms/step - loss: 0.3995 - accuracy: 0.8570 - val_loss: 0.4479 - val_accuracy: 0.8333

Epoch 00010: val_loss improved from 0.45283 to 0.44795, saving model to ./tmp/checkpoint_bahar_a5
INFO:tensorflow:Assets written to: ./tmp/checkpoint_bahar_a5/assets
Epoch 11/100
15/15 [==============================] - 0s 4ms/step - loss: 0.3790 - accuracy: 0.8504 - val_loss: 0.4514 - val_accuracy: 0.8333

Epoch 00011: val_loss did not improve from 0.44795
Epoch 12/100
15/15 [==============================] - 0s 5ms/step - loss: 0.3646 - accuracy: 0.8680 - val_loss: 0.4822 - val_accuracy: 0.8125

Epoch 00012: val_loss did not improve from 0.44795
Epoch 13/100
15/15 [==============================] - 0s 5ms/step - loss: 0.3408 - accuracy: 0.8812 - val_loss: 0.4482 - val_accuracy: 0.8125

Epoch 00013: val_loss did not improve from 0.44795
Epoch 14/100
15/15 [==============================] - 0s 5ms/step - loss: 0.3349 - accuracy: 0.8834 - val_loss: 0.4335 - v

15/15 [==============================] - 0s 5ms/step - loss: 0.0403 - accuracy: 0.9912 - val_loss: 0.9610 - val_accuracy: 0.7708

Epoch 00093: val_loss did not improve from 0.43351
Epoch 94/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0438 - accuracy: 0.9835 - val_loss: 0.8673 - val_accuracy: 0.7708

Epoch 00094: val_loss did not improve from 0.43351
Epoch 95/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0427 - accuracy: 0.9868 - val_loss: 0.9053 - val_accuracy: 0.7708

Epoch 00095: val_loss did not improve from 0.43351
Epoch 96/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0398 - accuracy: 0.9868 - val_loss: 0.9561 - val_accuracy: 0.7708

Epoch 00096: val_loss did not improve from 0.43351
Epoch 97/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0343 - accuracy: 0.9868 - val_loss: 1.0026 - val_accuracy: 0.7708

Epoch 00097: val_loss did not improve from 0.43351
Epoch 98/100
15/15 [=======================


Epoch 00035: val_loss did not improve from 0.43351
Epoch 36/100
8/8 [==============================] - 0s 6ms/step - loss: 0.2332 - accuracy: 0.9362 - val_loss: 0.5228 - val_accuracy: 0.8125

Epoch 00036: val_loss did not improve from 0.43351
Epoch 37/100
8/8 [==============================] - 0s 7ms/step - loss: 0.2273 - accuracy: 0.9263 - val_loss: 0.5253 - val_accuracy: 0.8333

Epoch 00037: val_loss did not improve from 0.43351
Epoch 38/100
8/8 [==============================] - 0s 7ms/step - loss: 0.2223 - accuracy: 0.9395 - val_loss: 0.5131 - val_accuracy: 0.7708

Epoch 00038: val_loss did not improve from 0.43351
Epoch 39/100
8/8 [==============================] - 0s 8ms/step - loss: 0.2200 - accuracy: 0.9307 - val_loss: 0.5199 - val_accuracy: 0.7708

Epoch 00039: val_loss did not improve from 0.43351
Epoch 40/100
8/8 [==============================] - 0s 8ms/step - loss: 0.2077 - accuracy: 0.9296 - val_loss: 0.5099 - val_accuracy: 0.8125

Epoch 00040: val_loss did not improve f

In [118]:
# in code 300 ta model deep learning ro train mikone ba batch size hA va epoc haye mokhtalf va to motuni in model ro azin be bad estefade koni 

In [123]:
import keras
model = keras.models.load_model('./tmp/checkpoint_bahar_a5/')

In [125]:
checkpoint_filepath = './tmp/checkpoint_amin_a/'

In [126]:
from sklearn.metrics import classification_report

# Load the trained model from the checkpoint
model = keras.models.load_model(checkpoint_filepath)

# Convert integer labels to one-hot encoded labels
y_test_encoded = to_categorical(y_test, num_classes=3)

# Predict labels for X_test_embeddings
y_pred_encoded = model.predict(X_test_embeddings)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred_encoded, axis=1)
y_test = np.argmax(y_test_encoded, axis=1)

# Generate a classification report
target_names = ['Class 0', 'Class 1', 'Class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.88      0.88      0.88         8
     Class 1       0.75      0.60      0.67        10
     Class 2       0.91      0.97      0.94        30

    accuracy                           0.88        48
   macro avg       0.84      0.81      0.83        48
weighted avg       0.87      0.88      0.87        48



In [120]:
from sklearn.metrics import classification_report

# Load the trained model from the checkpoint
model = keras.models.load_model(checkpoint_filepath)

# Convert integer labels to one-hot encoded labels
y_test_encoded = to_categorical(y_test, num_classes=3)

# Predict labels for X_test_embeddings
y_pred_encoded = model.predict(X_test_embeddings)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred_encoded, axis=1)
y_test = np.argmax(y_test_encoded, axis=1)

# Generate a classification report
target_names = ['Class 0', 'Class 1', 'Class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.83      0.62      0.71         8
     Class 1       0.56      0.50      0.53        10
     Class 2       0.85      0.93      0.89        30

    accuracy                           0.79        48
   macro avg       0.75      0.69      0.71        48
weighted avg       0.78      0.79      0.78        48



In [217]:
from sklearn.metrics import classification_report

# Load the trained model from the checkpoint
model = keras.models.load_model(checkpoint_filepath)

# Convert integer labels to one-hot encoded labels
y_test_encoded = to_categorical(y_test, num_classes=3)

# Predict labels for X_test_embeddings
y_pred_encoded = model.predict(X_test_embeddings)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred_encoded, axis=1)
y_test = np.argmax(y_test_encoded, axis=1)

# Generate a classification report
target_names = ['Class 0', 'Class 1', 'Class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.67      0.50      0.57         4
     Class 1       0.87      0.93      0.90        14
     Class 2       0.94      0.94      0.94        31

    accuracy                           0.90        49
   macro avg       0.82      0.79      0.80        49
weighted avg       0.89      0.90      0.89        49



In [178]:
from sklearn.metrics import classification_report

# Load the trained model from the checkpoint
model = keras.models.load_model(checkpoint_filepath)

# Convert integer labels to one-hot encoded labels
y_test_encoded = to_categorical(y_test, num_classes=3)

# Predict labels for X_test_embeddings
y_pred_encoded = model.predict(X_test_embeddings)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred_encoded, axis=1)
y_test = np.argmax(y_test_encoded, axis=1)

# Generate a classification report
target_names = ['Class 0', 'Class 1', 'Class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.77      0.91      0.83        11
     Class 1       0.50      0.43      0.46         7
     Class 2       0.90      0.87      0.89        31

    accuracy                           0.82        49
   macro avg       0.72      0.74      0.73        49
weighted avg       0.81      0.82      0.81        49



In [58]:
# pred_sigm= [1 if pred_ > thresh else 0 for pred_ in pred]

In [61]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.70      0.74        10
           1       0.67      0.50      0.57         8
           2       0.91      1.00      0.95        31

    accuracy                           0.86        49
   macro avg       0.79      0.73      0.75        49
weighted avg       0.84      0.86      0.85        49



In [60]:
confusion_matrix(y_test,pred_sigm)

NameError: name 'pred_sigm' is not defined

In [ ]:
fig, ax = plt.subplots()
#sns.heatmap(confusion_matrix(pred_soft[:,1], y_test_label[:,1]), annot= True)
sns.heatmap(confusion_matrix(y_test,pred_sigm),cmap="Blues", annot= True)
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
display(fig)

In [ ]:
plt.plot(histories[0].history['binary_accuracy'])
plt.plot(histories[0].history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
display(plt.show())
# summarize history for loss
plt.plot(histories[0].history['loss'])
plt.plot(histories[0].history['val_loss'])
plt.title('model loss')            



plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
display(plt.show())